# Harris Corners

Source: https://docs.opencv.org/3.4/d4/d7d/tutorial_harris_detector.html

Conclusion: Using the Harris method on a plain binarized image appears to be best for finding the corners, and I don't yet understand these fully.


In [1]:
import cv2
import numpy as np

img = cv2.imread("deskewed/1.jpg", 0)
thresh = 255

# Detector parameters
blockSize = 2
apertureSize = 3
k = 0.04

# Detecting corners
harris = cv2.cornerHarris(img, blockSize, apertureSize, k)

# Normalizing
normalized = np.empty(harris.shape, dtype=np.float32)
cv2.normalize(harris, normalized, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
scaled = cv2.convertScaleAbs(normalized)

# Drawing a circle around corners
for i in range(normalized.shape[0]):
    for j in range(normalized.shape[1]):
        if int(normalized[i,j]) > thresh:
            cv2.circle(scaled, (j,i), 5, (0), 2)

cv2.imwrite("corners/harris.jpg", scaled)

True

### Harris Corners

...on original
![](corners/harris.jpg)

In [2]:
# Detecting corners
gaussian_blur = cv2.GaussianBlur(img,(5,5),0)
_, otsu_after_gaussian = cv2.threshold(gaussian_blur, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
harris_otsu_after_gaussian = cv2.cornerHarris(otsu_after_gaussian, blockSize, apertureSize, k)

# Normalizing
otsu_normalized = np.empty(harris_otsu_after_gaussian.shape, dtype=np.float32)
cv2.normalize(harris_otsu_after_gaussian, otsu_normalized, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
otsu_scaled = cv2.convertScaleAbs(otsu_normalized)

# Drawing a circle around corners
for i in range(otsu_normalized.shape[0]):
    for j in range(otsu_normalized.shape[1]):
        if int(otsu_normalized[i,j]) > thresh:
            cv2.circle(otsu_scaled, (j,i), 5, (0), 2)
            
cv2.imwrite("corners/harris_otsu_after_gaussian.jpg", otsu_scaled)

True

...on otsu after gaussian
![](corners/harris_otsu_after_gaussian.jpg)

In [3]:
# Detecting corners
_, binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
harris_after_binary_threshold = cv2.cornerHarris(binary, blockSize, apertureSize, k)

# Normalizing
binary_normalized = np.empty(harris_after_binary_threshold.shape, dtype=np.float32)
cv2.normalize(harris_after_binary_threshold, binary_normalized, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
binary_scaled = cv2.convertScaleAbs(binary_normalized)

# Drawing a circle around corners
for i in range(binary_normalized.shape[0]):
    for j in range(binary_normalized.shape[1]):
        if int(binary_normalized[i,j]) > thresh:
            cv2.circle(binary_scaled, (j,i), 5, (0), 2)
            
cv2.imwrite("corners/harris_after_binary_threshold.jpg", binary_scaled)


True

...on plain binary threshold
![](corners/harris_after_binary_threshold.jpg)

### Shi-Tomasi Corners

In [4]:
import random as rng

rng.seed(12345)
maxCorners = 23 # initial threshold

# Parameters for Shi-Tomasi algorithm
qualityLevel = 0.01
minDistance = 10

# Copy the source image
copy = np.copy(img)

# Apply corner detection
corners = cv2.goodFeaturesToTrack(img, maxCorners, qualityLevel, minDistance, None, \
    blockSize=3, gradientSize=3, useHarrisDetector=False, k=0.04)

# Draw corners detected
print('** Number of corners detected:', corners.shape[0])
radius = 4
for i in range(corners.shape[0]):
    cv2.circle(copy, (corners[i,0,0], corners[i,0,1]), radius, \
        (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256)), cv2.FILLED)
    
cv2.imwrite("corners/shi_tomasi.jpg", copy)

** Number of corners detected: 23


<ipython-input-4-17ce61acad3d>:21: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  cv2.circle(copy, (corners[i,0,0], corners[i,0,1]), radius, \


True

... on original
![](corners/shi_tomasi.jpg)